# Environment setup

In [1]:
import detectron2.utils.comm as comm
from detectron2.config import get_cfg
from detectron2.projects.deeplab import add_deeplab_config
from detectron2.utils.logger import setup_logger

from MODULES.MaskFormer.config import add_mask_former_config

import torch
import matplotlib.pyplot as plt

In [2]:
# Create configs and perform basic setups

cfg = get_cfg()
add_deeplab_config(cfg)
add_mask_former_config(cfg)
cfg.set_new_allowed(True)
cfg.merge_from_file("MODULES/MaskFormer/configs/custom/MaskAVSL_R50.yaml")
# cfg.merge_from_list(args.opts)
cfg.MODEL.DEVICE = "cuda:1"
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.eval_only = True
cfg.freeze()
# default_setup(cfg, args)
# Setup logger for "mask_former" module
setup_logger(output=cfg.OUTPUT_DIR, distributed_rank=comm.get_rank(), name="MaskAVSL")

<Logger MaskAVSL (DEBUG)>

# ResNet50

In [ ]:
# Create configs and perform basic setups

cfg = get_cfg()
add_deeplab_config(cfg)
add_mask_former_config(cfg)
cfg.set_new_allowed(True)
cfg.merge_from_file("MODULES/MaskFormer/configs/custom/MaskAVSL_R50.yaml")
# cfg.merge_from_list(args.opts)
cfg.MODEL.DEVICE = "cuda:1"
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.eval_only = True
cfg.freeze()
# default_setup(cfg, args)
# Setup logger for "mask_former" module
setup_logger(output=cfg.OUTPUT_DIR, distributed_rank=comm.get_rank(), name="MaskAVSL")

<Logger MaskAVSL (DEBUG)>

In [ ]:
from detectron2.modeling import build_backbone

resnet50 = build_backbone(cfg)

In [ ]:
import torch
resnet50.to(torch.device(cfg.MODEL.DEVICE)) # 1225 Mb

ResNet(
  (stem): BasicStem(
    (conv1): Conv2d(
      3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
      (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
    )
  )
  (res2): Sequential(
    (0): BottleneckBlock(
      (shortcut): Conv2d(
        64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
        (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
      )
      (conv1): Conv2d(
        64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
      (conv2): Conv2d(
        64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
      (conv3): Conv2d(
        64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
        (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
      )
    )
    (1): BottleneckBlock(
      (conv1): Conv2d(
        256, 64, kernel_size=(1, 1), stride=(1, 1), bias=Fa

In [ ]:
%cd MODULES/ckpt

/workspace/GitHub/AVSL/MODULES/ckpt


In [ ]:
!wget https://download.pytorch.org/models/resnet50-19c8e357.pth -O r50.pth
!python tool/convert-torchvision-to-d2.py r50.pth R-50.pkl

conv1.weight -> stem.conv1.weight
bn1.running_mean -> stem.conv1.norm.running_mean
bn1.running_var -> stem.conv1.norm.running_var
bn1.weight -> stem.conv1.norm.weight
bn1.bias -> stem.conv1.norm.bias
layer1.0.conv1.weight -> res2.0.conv1.weight
layer1.0.bn1.running_mean -> res2.0.conv1.norm.running_mean
layer1.0.bn1.running_var -> res2.0.conv1.norm.running_var
layer1.0.bn1.weight -> res2.0.conv1.norm.weight
layer1.0.bn1.bias -> res2.0.conv1.norm.bias
layer1.0.conv2.weight -> res2.0.conv2.weight
layer1.0.bn2.running_mean -> res2.0.conv2.norm.running_mean
layer1.0.bn2.running_var -> res2.0.conv2.norm.running_var
layer1.0.bn2.weight -> res2.0.conv2.norm.weight
layer1.0.bn2.bias -> res2.0.conv2.norm.bias
layer1.0.conv3.weight -> res2.0.conv3.weight
layer1.0.bn3.running_mean -> res2.0.conv3.norm.running_mean
layer1.0.bn3.running_var -> res2.0.conv3.norm.running_var
layer1.0.bn3.weight -> res2.0.conv3.norm.weight
layer1.0.bn3.bias -> res2.0.conv3.norm.bias
layer1.0.downsample.0.weight -> res

In [ ]:
%cd ../../

/workspace/GitHub/AVSL


In [ ]:
import torch
weights = torch.load("MODULES/ckpt/R-50.pkl")

In [ ]:
resnet50.load_state_dict(weights["model"])

TypeError: 'int' object is not callable

In [ ]:
# for param_tensor in resnet50.state_dict():
#     print(param_tensor, "\t", resnet50.state_dict()[param_tensor].size())

In [ ]:
from torchsummary import summary

summary(resnet50, (3, 640, 640), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
 FrozenBatchNorm2d-1         [-1, 64, 320, 320]             128
            Conv2d-2         [-1, 64, 320, 320]           9,408
         BasicStem-3         [-1, 64, 160, 160]               0
 FrozenBatchNorm2d-4         [-1, 64, 160, 160]             128
            Conv2d-5         [-1, 64, 160, 160]           4,096
 FrozenBatchNorm2d-6         [-1, 64, 160, 160]             128
            Conv2d-7         [-1, 64, 160, 160]          36,864
 FrozenBatchNorm2d-8        [-1, 256, 160, 160]             512
            Conv2d-9        [-1, 256, 160, 160]          16,384
FrozenBatchNorm2d-10        [-1, 256, 160, 160]             512
           Conv2d-11        [-1, 256, 160, 160]          16,384
  BottleneckBlock-12        [-1, 256, 160, 160]               0
FrozenBatchNorm2d-13         [-1, 64, 160, 160]             128
           Conv2d-14         [-1, 64, 1

In [ ]:
param_size = 0
for param in resnet50.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in resnet50.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('resnet50 size: {:.3f}MB'.format(size_all_mb))

backbone size: 89.879MB


In [ ]:
resnet50.state_dict()

OrderedDict([('stem.conv1.weight',
              tensor([[[[ 4.7298e-02,  1.7676e-02, -1.3182e-02,  ..., -1.5572e-02,
                          3.0704e-02,  2.2820e-02],
                        [ 1.6432e-02, -1.5360e-02, -1.9780e-02,  ..., -3.7262e-02,
                         -8.8089e-03,  3.4164e-02],
                        [-1.1109e-03,  2.1470e-03,  1.1781e-03,  ...,  3.5993e-02,
                         -3.6558e-02,  4.8545e-03],
                        ...,
                        [ 3.0532e-04,  6.1797e-02, -1.5987e-02,  ..., -2.1682e-02,
                          3.4853e-02,  7.1491e-03],
                        [-4.6115e-02, -7.1832e-04,  1.0117e-02,  ..., -7.8956e-03,
                          3.2217e-02,  1.2048e-02],
                        [ 6.4859e-02,  4.2602e-03,  5.6116e-03,  ..., -2.9466e-03,
                         -2.7983e-02, -2.8463e-03]],
              
                       [[ 2.0348e-02, -2.1133e-02, -1.7321e-02,  ..., -1.6096e-03,
                         -5

# ResNet101

In [ ]:
# Create configs and perform basic setups

cfg = get_cfg()
add_deeplab_config(cfg)
add_mask_former_config(cfg)
cfg.set_new_allowed(True)
cfg.merge_from_file("MODULES/MaskFormer/configs/custom/MaskAVSL_R101.yaml")
# cfg.merge_from_list(args.opts)
cfg.MODEL.DEVICE = "cuda:1"
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.eval_only = True
cfg.freeze()
# default_setup(cfg, args)
# Setup logger for "mask_former" module
setup_logger(output=cfg.OUTPUT_DIR, distributed_rank=comm.get_rank(), name="MaskAVSL")

<Logger MaskAVSL (DEBUG)>

In [ ]:
from detectron2.modeling import build_backbone

resnet101 = build_backbone(cfg)

In [ ]:
import torch
resnet101.to(torch.device(cfg.MODEL.DEVICE)) # 1279 Mb

ResNet(
  (stem): DeepLabStem(
    (conv1): Conv2d(
      3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
      (norm): FrozenBatchNorm2d(num_features=32, eps=1e-05)
    )
    (conv2): Conv2d(
      32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (norm): FrozenBatchNorm2d(num_features=32, eps=1e-05)
    )
    (conv3): Conv2d(
      32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
    )
  )
  (res2): Sequential(
    (0): BottleneckBlock(
      (shortcut): Conv2d(
        64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
        (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
      )
      (conv1): Conv2d(
        64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
      (conv2): Conv2d(
        64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (norm

In [ ]:
for param_tensor in resnet101.state_dict():
    print(param_tensor, "\t", resnet101.state_dict()[param_tensor].size())

stem.conv1.weight 	 torch.Size([32, 3, 3, 3])
stem.conv1.norm.weight 	 torch.Size([32])
stem.conv1.norm.bias 	 torch.Size([32])
stem.conv1.norm.running_mean 	 torch.Size([32])
stem.conv1.norm.running_var 	 torch.Size([32])
stem.conv2.weight 	 torch.Size([32, 32, 3, 3])
stem.conv2.norm.weight 	 torch.Size([32])
stem.conv2.norm.bias 	 torch.Size([32])
stem.conv2.norm.running_mean 	 torch.Size([32])
stem.conv2.norm.running_var 	 torch.Size([32])
stem.conv3.weight 	 torch.Size([64, 32, 3, 3])
stem.conv3.norm.weight 	 torch.Size([64])
stem.conv3.norm.bias 	 torch.Size([64])
stem.conv3.norm.running_mean 	 torch.Size([64])
stem.conv3.norm.running_var 	 torch.Size([64])
res2.0.shortcut.weight 	 torch.Size([256, 64, 1, 1])
res2.0.shortcut.norm.weight 	 torch.Size([256])
res2.0.shortcut.norm.bias 	 torch.Size([256])
res2.0.shortcut.norm.running_mean 	 torch.Size([256])
res2.0.shortcut.norm.running_var 	 torch.Size([256])
res2.0.conv1.weight 	 torch.Size([64, 64, 1, 1])
res2.0.conv1.norm.weight 	

In [ ]:
from torchsummary import summary

summary(resnet101, (3, 640, 640), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
 FrozenBatchNorm2d-1         [-1, 32, 320, 320]              64
            Conv2d-2         [-1, 32, 320, 320]             864
 FrozenBatchNorm2d-3         [-1, 32, 320, 320]              64
            Conv2d-4         [-1, 32, 320, 320]           9,216
 FrozenBatchNorm2d-5         [-1, 64, 320, 320]             128
            Conv2d-6         [-1, 64, 320, 320]          18,432
       DeepLabStem-7         [-1, 64, 160, 160]               0
 FrozenBatchNorm2d-8         [-1, 64, 160, 160]             128
            Conv2d-9         [-1, 64, 160, 160]           4,096
FrozenBatchNorm2d-10         [-1, 64, 160, 160]             128
           Conv2d-11         [-1, 64, 160, 160]          36,864
FrozenBatchNorm2d-12        [-1, 256, 160, 160]             512
           Conv2d-13        [-1, 256, 160, 160]          16,384
FrozenBatchNorm2d-14        [-1, 256, 1

In [ ]:
resnet101.state_dict()

OrderedDict([('stem.conv1.weight',
              tensor([[[[ 2.7150e-02,  4.0202e-02,  3.3753e-02,  ..., -3.4984e-02,
                         -7.8680e-03, -2.0802e-02],
                        [-1.7525e-02,  2.0183e-02, -1.0996e-02,  ..., -3.1687e-02,
                         -2.3135e-02,  2.8585e-02],
                        [ 2.6273e-02, -3.1678e-02,  2.7108e-02,  ..., -4.5408e-02,
                         -9.0770e-03,  1.3524e-02],
                        ...,
                        [ 3.7497e-02, -4.9371e-02,  5.4058e-02,  ..., -3.5271e-02,
                          2.7010e-02, -2.1936e-02],
                        [-2.0629e-02, -2.2742e-02,  2.2015e-02,  ..., -3.1601e-03,
                         -4.0475e-02,  4.1254e-03],
                        [-1.0298e-02, -2.2968e-02,  5.9657e-03,  ...,  3.3392e-02,
                          2.3562e-03, -1.9873e-02]],
              
                       [[-2.0070e-02,  3.2798e-02, -3.3159e-02,  ..., -2.7945e-02,
                          3

# Swin-base

In [2]:
# Create configs and perform basic setups

cfg = get_cfg()
add_deeplab_config(cfg)
add_mask_former_config(cfg)
cfg.set_new_allowed(True)
cfg.merge_from_file("MODULES/MaskFormer/configs/custom/MaskAVSL_swin_base.yaml")
# cfg.merge_from_list(args.opts)
cfg.MODEL.DEVICE = "cuda:1"
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.eval_only = True
cfg.freeze()
# default_setup(cfg, args)
# Setup logger for "mask_former" module
setup_logger(output=cfg.OUTPUT_DIR, distributed_rank=comm.get_rank(), name="MaskAVSL")

Loading config MODULES/MaskFormer/configs/custom/MaskAVSL_swin_base.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


<Logger MaskAVSL (DEBUG)>

In [3]:
from MODULES.build import build_swin_backbone

swin = build_swin_backbone(cfg, cfg.MODEL.PIXEL_MEAN, cfg.MODEL.WEIGHTS)

/opt/conda/envs/imsi3.8/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272068694/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
import torch
swin.to(torch.device(cfg.MODEL.DEVICE)) # 1459 Mb

D2SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): BasicLayer(
      (blocks): ModuleList(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=128, out_features=384, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=128, out_features=128, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path): Identity()
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=128, out_features=512, bias=True)
            (act): GELU()
            (fc2): Linear(i

In [ ]:
from torchsummary import summary

summary(swin, (3, 640, 640), device='cpu')

AttributeError: 'int' object has no attribute 'size'

# Swin-large

In [2]:
# Create configs and perform basic setups

cfg = get_cfg()
add_deeplab_config(cfg)
add_mask_former_config(cfg)
cfg.set_new_allowed(True)
cfg.merge_from_file("MODULES/MaskFormer/configs/custom/MaskAVSL_swin_large.yaml")
# cfg.merge_from_list(args.opts)
cfg.MODEL.DEVICE = "cuda:1"
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.eval_only = True
cfg.freeze()
# default_setup(cfg, args)
# Setup logger for "mask_former" module
setup_logger(output=cfg.OUTPUT_DIR, distributed_rank=comm.get_rank(), name="MaskAVSL")

Loading config MODULES/MaskFormer/configs/custom/MaskAVSL_swin_base.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


<Logger MaskAVSL (DEBUG)>

In [14]:
from MODULES.build import build_swin_backbone

swin = build_swin_backbone(cfg, cfg.MODEL.PIXEL_MEAN, cfg.MODEL.WEIGHTS)

# Train

In [4]:
from DATALOADER import VideoDataLoader

folder_path = 'DATA/videos'
dataloader = VideoDataLoader(cfg, folder_path)
print(len(dataloader))

# detectron2/engine/train_loop.py
data = next(iter(dataloader))

221831 of train videos have loaded
55458


In [ ]:
from MODULES.build import build_swin_backbone, build_pixel_decoder, build_mask_predictor
from detectron2.modeling import build_backbone

import torch

backbone = build_swin_backbone(cfg, cfg.MODEL.PIXEL_MEAN, cfg.MODEL.WEIGHTS)
# backbone = build_backbone(cfg)
backbone.to(torch.device(cfg.MODEL.DEVICE)) # 1225

In [ ]:
pixel_decoder = build_pixel_decoder(cfg, backbone.output_shape())
pixel_decoder.to(torch.device(cfg.MODEL.DEVICE)) # 1225 -> 1253
mask_predictor = build_mask_predictor(cfg, cfg.MODEL.SEM_SEG_HEAD.CONVS_DIM)
mask_predictor.to(torch.device(cfg.MODEL.DEVICE)) # 1225 -> 1599
print()

In [ ]:
from detectron2.structures import ImageList

pixel_mean = torch.Tensor(cfg.MODEL.PIXEL_MEAN).view(-1, 1, 1)
pixel_std = torch.Tensor(cfg.MODEL.PIXEL_STD).view(-1, 1, 1)

images = [x for x in data["image"]]
images = [(x - pixel_mean) / pixel_std for x in images]
images = ImageList.from_tensors(images, cfg.MODEL.MASK_FORMER.SIZE_DIVISIBILITY)

In [ ]:
with torch.no_grad():
    image_feature = backbone(images.tensor.to(cfg.MODEL.DEVICE)) # 1599 -> 2211
pp_embeds, image_features = pixel_decoder(image_feature) # 2211 -> 3463

import librosa
import numpy as np

audio_list = []
for path in data['mixed_audio_path']:
    mixed_audio, sr = librosa.load(path, mono=True, sr=16000)
    mixed_audio = np.append(mixed_audio, np.zeros(3000))
    mixed_audio = mixed_audio[:160000]
    audio_list.append(torch.from_numpy(mixed_audio).unsqueeze(1))
mixed_audio = ImageList.from_tensors(audio_list).tensor
mixed_audio = mixed_audio.type(torch.cuda.FloatTensor)
del audio_list

outputs = mask_predictor(image_features, pp_embeds, mixed_audio.to(cfg.MODEL.DEVICE)) # 3463 -> 9793

Calling forward() may cause unpredicted behavior of PixelDecoder module.


In [ ]:
for k in outputs.keys():
    print(k, outputs[k].shape)

pp_embeds torch.Size([4, 256, 160, 160])
audio_sep_tokens torch.Size([4, 4, 128])
mixed_audio_spec torch.Size([4, 501, 1025])
sep_audio_wavs torch.Size([20, 160000, 1])
sep_audio_specs torch.Size([20, 1, 501, 1025])
sep_audio_specs_wo_noise torch.Size([4, 4, 501, 1025])
sep_audio_wavs_wo_noise torch.Size([4, 4, 160000])
sep_audio_features torch.Size([4, 4, 128])
sep_audio_features_comp torch.Size([4, 4, 128])
sep_audio_features_embeds torch.Size([4, 4, 256])
pred_masks torch.Size([4, 4, 160, 160])


# feature extraction

In [3]:
from DATALOADER import ImageDataLoader

folder_path = 'DATA/images'
image_dataloader = ImageDataLoader(cfg, folder_path, 1)
print(len(image_dataloader))

data = next(iter(image_dataloader))

7428 of train videos have loaded
7428


In [4]:
from MODULES.AVSL_model import AVSLModel

model = AVSLModel(cfg, training=False)

In [10]:
# save as pkl

from tqdm.auto import tqdm
import pickle

features = {}
for i, batch in tqdm(enumerate(image_dataloader)):
    img_name = batch["image_name"][0][:-4]
    features[img_name] = model.feature_extraction(batch)
    # file_name = batch["image_name"][0][:-4]
    # with open(f"DATA/image_features/{file_name}_feature.pkl", "wb") as f:
    #     pickle.dump(image_feature, f)
    if i == 5:
        break

0it [00:00, ?it/s]

In [14]:
with open(f"DATA/image_features/features.pkl", "wb") as f:
    pickle.dump(features, f)

In [ ]:
with open(f"DATA/image_features/features.pkl", "rb") as f:
    feature = pickle.load(f)
print(feature)

In [5]:
# save only res5 tensro as torch.save (pt file)

from tqdm.auto import tqdm

for i, batch in tqdm(enumerate(image_dataloader)):
    img_name = batch["image_name"][0][:-4]
    feature = model.feature_extraction(batch)
    torch.save(feature, f"DATA/image_features/{img_name}_feature.pt")
    break

0it [00:00, ?it/s]

In [64]:
# save as json file

import json

json.dumps(image_feature)

TypeError: Object of type Tensor is not JSON serializable